<a href="https://colab.research.google.com/github/Hani-kanaan/Adatpers_On_Demand/blob/main/adapters_on_demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

hanikanaan774_google_cluster_sample_path = kagglehub.dataset_download('hanikanaan774/google-cluster-sample')
hanikanaan774_prompts_path = kagglehub.dataset_download('hanikanaan774/prompts')

print('Data source import complete.')


In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers peft accelerate
!pip install -U accelerate peft
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126
!pip install transformers peft accelerate datasets
!pip install safetensors


In [ ]:
!pip uninstall sentence-transformers -y

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import torch

torch.cuda.empty_cache()

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

#linux quantization:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_cfg,
    #torch_dtype=torch.float16,
    quantization_config= bnb_cfg ,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
#adapter from: https://huggingface.co/DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927
# from peft import PeftModel

# model = PeftModel.from_pretrained(
#     base_model,
#     "DreamGallery/Qwen-Qwen2.5-1.5B-Instruct-1727452927",  # adapter path
#     trust_remote_code=True
# )


In [ ]:
prompt = "Explain the difference between L1 and L2 regularization in machine learning."
inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
with torch.no_grad():
    out = base_model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))


In [ ]:
#dataset of prompts:
from datasets import load_dataset

ds1 = load_dataset("fka/awesome-chatgpt-prompts")

In [ ]:
from datasets import load_dataset

ds = load_dataset("RUC-DataLab/DataScience-Instruct-500K")

In [ ]:
from datasets import load_dataset

ds2 = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

In [ ]:
import datasets
print(datasets.__version__)

In [ ]:
def tokenize_ds(batch):
    prompts = []
    targets = []

    for messages, evaluation in zip(batch["messages"], batch["evaluation"]):
        if isinstance(messages, list):
            prompt = "\n".join([f"{m.get('role', '')}: {m.get('content', '')}" for m in messages])
        else:
            prompt = str(messages)

        prompts.append(prompt)
        targets.append(str(evaluation))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def tokenize_ds1(batch):
    prompts = []
    targets = []

    for act, prompt in zip(batch["act"], batch["prompt"]):
        inst = " ".join(act) if isinstance(act, list) else str(act)
        prompts.append(inst)
        targets.append(str(prompt))

    return tokenizer(
        prompts,
        text_target=targets,
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def tokenize_ds3(batch):
    return tokenizer(
        batch["Question"],
        text_target=batch["Response"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )



train1 = ds.map(tokenize_ds, batched=True, batch_size=256)
train2 = ds1.map(tokenize_ds1, batched=True, batch_size=256)



In [ ]:
train3 = ds2.map(tokenize_ds3, batched=False)

In [ ]:
from peft import LoraConfig
lora_cfg_1 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)
lora_cfg_2 = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    task_type="CAUSAL_LM"
)


In [ ]:

print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

In [ ]:
!nvidia-smi

In [ ]:
from peft import get_peft_model
from transformers import TrainingArguments, Trainer
torch.cuda.empty_cache()

model1 = get_peft_model(base_model, lora_cfg_1)
args1 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter1",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train1 = train1["train"].select(range(100))  # use only first 100 samples for faster runtime

trainer1 = Trainer(model=model1, args=args1, train_dataset=small_train1)
trainer1.train()
model1.save_pretrained("./adapters/adapter1")

del model1
torch.cuda.empty_cache()

model2 = get_peft_model(base_model, lora_cfg_2)
args2 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter2",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train2 = train2["train"].select(range(100))
trainer2 = Trainer(model=model2, args=args2, train_dataset=small_train2)
trainer2.train()
model2.save_pretrained("./adapters/adapter2")


#adapter 3:

model3 = get_peft_model(base_model, lora_cfg_2)
args3 = TrainingArguments(
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=2e-4,
    output_dir="./adapter3",
    gradient_accumulation_steps=1,
    logging_dir="./logs",         # where logs are saved
    logging_steps=5,              # print every 5 steps
    report_to="none",             # disable WandB, keep console output
    disable_tqdm=False            # show progress bar
)
small_train3 = train3["train"].select(range(100))
trainer3 = Trainer(model=model3, args=args3, train_dataset=small_train3)
trainer3.train()
model3.save_pretrained("./adapters/adapter3")


In [ ]:
import os
from safetensors.torch import load_file

def get_dir_size_mb(path):
    total_size = 0
    for root, _, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            if os.path.isfile(fp):
                total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # MB

def get_adapter_vram_mb(adapter_path, dtype_bytes=2):
    """
    dtype_bytes:
        fp16 / bf16 = 2
        fp32 = 4
    """
    adapter_file = os.path.join(adapter_path, "adapter_model.safetensors")
    state_dict = load_file(adapter_file)  # SAFE loader
    total_params = sum(t.numel() for t in state_dict.values())
    vram_mb = total_params * dtype_bytes / (1024 * 1024)
    return vram_mb

# Configuration
base_path = "/kaggle/working/adapters"
adapter_names = ["adapter1", "adapter2", "adapter3"]

# Process each adapter
print("=" * 55)
print("ADAPTER SIZE ANALYSIS")
print("=" * 55)

for adapter_name in adapter_names:
    adapter_path = os.path.join(base_path, adapter_name)

    print(f"\n📦 {adapter_name}")
    print("-" * 30)

    if os.path.exists(adapter_path):
        # Disk size
        size_mb = get_dir_size_mb(adapter_path)
        print(f"  💾 Disk size:   {size_mb:.2f} MB")

        # VRAM size
        vram_cost = get_adapter_vram_mb(adapter_path, dtype_bytes=2)
        print(f" VRAM cost:   {vram_cost:.2f} MB")
    else:
        print(f" Directory not found: {adapter_path}")

print("\n" + "=" * 55)

In [ ]:
from transformers import  AutoTokenizer
from peft import PeftModel

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_path = "./adapters/adapter1"

tokenizer = AutoTokenizer.from_pretrained(model_name)


model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

text ="""### Instruction:
explain the difference between L1 and L2 regularization in machine learning.

### Input:

### Response:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7, top_p=0.9)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded)

In [ ]:
!pip install --upgrade --force-reinstall transformers sentence-transformers -q

# Restart kernel after this
import transformers


In [ ]:
import transformers

!pip list | grep -E "transformers|sentence"
!pip install transformers==4.57.1 sentence-transformers==2.2.2 -q


In [ ]:

from sentence_transformers import SentenceTransformer , util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
adapter_tasks = [
    "role-playing instruction following across many creative and utility roles",
    "data science and machine learning problem solving with step by step reasoning",
    "medical diagnosis and clinical reasoning with detailed chain-of thought explanations"
]
adapter_embs = model.encode(adapter_tasks)

print(adapter_embs)
def select_adapter(user_prompt):
    prompt_emb = model.encode([user_prompt], normalize_embeddings=True)[0]
    sims = util.cos_sim(prompt_emb, adapter_embs)[0]
    top = sims.argmax().item()
    best_score = sims[top].item()
    return adapter_tasks[top], best_score
task, score = select_adapter("Can you summarize this text?")
print(task, score)


In [ ]:
prompt = "medical terms prompt"
adapter, scores = select_adapter(prompt)
print(adapter)


In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/google-cluster-sample/borg_traces_data.csv')
prompts_df = pd.read_excel('/kaggle/input/prompts/Prompts.xlsx')
df["collection_id"] = df["collection_id"].astype(int) % 3
df.head()
prompts = prompts_df['Prompt'].dropna().tolist()



prompts_to_assign = []
for i in range(len(df)):
    # Cycle through prompts if needed
    prompt_idx = i % len(prompts)
    prompts_to_assign.append(prompts[prompt_idx])

df['collection_type'] = prompts_to_assign
df["collection_id"] = df["collection_id"].astype(int) % 3
print(f"\nUpdated dataset shape: {df.shape}")
print("\nFirst 5 rows with new collection_type:")
print(df[['time', 'collection_id', 'collection_type', 'assigned_memory']].head())

In [ ]:
print(prompts_to_assign)

In [ ]:
import gymnasium as gym
from collections import defaultdict, deque
import numpy as np
import pandas as pd
from collections import defaultdict
import pandas as pd
import os
class AdapterEnv(gym.Env):

    def __init__(self, arrivals_by_time, num_adapters=3, num_gpus=2, max_steps=200):
        super().__init__()
        self.num_adapters = num_adapters
        self.num_gpus = num_gpus
        self.max_steps = max_steps
        self.arrivals_by_time = arrivals_by_time
        self.max_time = max(arrivals_by_time.keys()) if arrivals_by_time else 0

        # Adjusted adapter sizes - make adapter 2 more competitive
        self.adapter_vram = np.array([0.3, 1,1])  # 30%, 40%, 50%

        # Action space
        self.action_space = gym.spaces.Discrete(4 * num_adapters * num_gpus)

        # State space
        self.observation_space = gym.spaces.Tuple((
            gym.spaces.MultiBinary(num_gpus * num_adapters),
            gym.spaces.Box(low=0.0, high=1.0, shape=(num_gpus,)),
            gym.spaces.Discrete(100),
            gym.spaces.Discrete(num_adapters + 1)
        ))

        # Track adapter usage
        self.adapter_usage_stats = {0: 0, 1: 0, 2: 0}
        self.adapter_load_attempts = {0: 0, 1: 0, 2: 0}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.adapter_loaded = np.zeros((self.num_gpus, self.num_adapters), dtype=int)
        self.free_vram = np.ones(self.num_gpus)
        self.queue = deque()
        self.time = 0
        self.step_count = 0
        self.total_reward = 0.0

        self._arrival_step()
        return self._get_state(), {}

    def _arrival_step(self):

        if self.time in self.arrivals_by_time:
            self.queue.extend(self.arrivals_by_time[self.time])



    def _get_state(self):
        next_adapter_id = self.queue[0]["adapter_id"] if self.queue else self.num_adapters
        return (
            tuple(self.adapter_loaded.flatten()),
            tuple(self.free_vram),
            min(len(self.queue), 99),
            next_adapter_id
        )

    def step(self, action):
        # Decode action
        action_type = action // (self.num_adapters * self.num_gpus)
        rem = action % (self.num_adapters * self.num_gpus)
        adapter_id = rem // self.num_gpus
        gpu_id = rem % self.num_gpus

        reward = -0.01  # Smaller time penalty
        processed_task = False

        # Track which adapter is being targeted
        self.adapter_load_attempts[adapter_id] += 1

        # ---- LOAD (type 0) ----
        if action_type == 0:
            cost = self.adapter_vram[adapter_id]
            if self.adapter_loaded[gpu_id, adapter_id] == 0 and self.free_vram[gpu_id] >= cost:
                self.adapter_loaded[gpu_id, adapter_id] = 1
                self.free_vram[gpu_id] -= cost

                if self.queue and self.queue[0]["adapter_id"] == adapter_id:
                    reward = 0.3
                else:
                    reward = -0.05   # discourage speculative loads


        # ---- EVICT (type 1) ----
        elif action_type == 1:
            if self.adapter_loaded[gpu_id, adapter_id] == 1:
                self.adapter_loaded[gpu_id, adapter_id] = 0
                self.free_vram[gpu_id] += self.adapter_vram[adapter_id]

                if self.free_vram[gpu_id] < 0.2:
                    reward += 0.4   # emergency eviction
                else:
                    reward -= 0.05  # pointless eviction


        # ---- REUSE (type 2) ----
        elif action_type == 2:
            if self.queue:
                task = self.queue[0]
                if (task["adapter_id"] == adapter_id and
                    self.adapter_loaded[gpu_id, adapter_id] == 1):
                    self.queue.popleft()
                    processed_task = True
                    reward = 1.0
                else:
                    reward = -1.0

        # ---- SEND (type 3) ----
        elif action_type == 3:
            if self.queue:
                task = self.queue[0]
                if (task["adapter_id"] == adapter_id and
                    self.adapter_loaded[gpu_id, adapter_id] == 1):
                    self.queue.popleft()
                    processed_task = True

                    # ADJUSTED: Higher reward for sending larger adapters
                    # base_reward = 0.8
                    # size_bonus = 0.4 * self.adapter_vram[adapter_id]
                    # reward += base_reward + size_bonus
                    reward = 0.8

        # Advance simulation
        self.time += 1
        self._arrival_step()
        self.step_count += 1
        self.total_reward += reward

        terminated = (
            self.step_count >= self.max_steps or
            (not self.queue and self.time > self.max_time)
        )

        info = {
            'processed': processed_task,
            'adapter_stats': self.adapter_usage_stats.copy(),
            'load_attempts': self.adapter_load_attempts.copy()
        }

        return self._get_state(), reward, terminated, False, info


def train_q_learning(env, episodes=5000):
    """Q-learning training loop"""


    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    alpha = 0.1
    gamma = 0.95
    epsilon = 0.2
    epsilon_decay = 0.995

    rewards_history = []
    epsilon_history = []
    for ep in range(episodes):
        state, _ = env.reset()
        episode_reward = 0

        while True:
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q[state])

            next_state, reward, done, _, _ = env.step(action)
            episode_reward += reward

            Q[state][action] += alpha * (
                reward + gamma * np.max(Q[next_state]) - Q[state][action]
            )

            state = next_state

            if done:
                break

        rewards_history.append(episode_reward)
        epsilon_history.append(epsilon)
        epsilon *= epsilon_decay

        if ep % 500 == 0:
            avg = np.mean(rewards_history[-500:])
            print(f"Ep {ep:4d} | Avg Reward: {avg:6.2f} | Epsilon: {epsilon:.3f}")

    return Q, rewards_history , epsilon_history

def test_env_heuristic(env, max_steps=200):
    """Test with greedy heuristic policy """
    state, _ = env.reset()

    A = env.num_adapters * env.num_gpus
    G = env.num_gpus

    total_reward = 0

    print(f"\n=== GOOGLE TRACE HEURISTIC TEST ===")
    print(f"Start: Queue={len(env.queue) if hasattr(env, 'queue') else state[2]} | "
          f"VRAM: {[f'{v:.2f}' for v in state[1]]}")
    print("-" * 60)

    for step in range(max_steps):
        action = 0
        action_type_name = "WAIT"
        adapter_id = 0
        gpu_id = 0

        if env.queue:
            next_adapter = env.queue[0]['adapter_id']
            loaded = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)

            # 1. SEND if possible
            sent = False
            for g in range(env.num_gpus):
                if loaded[g, next_adapter] == 1:
                    action = 3 * A + next_adapter * G + g
                    action_type_name = "SEND"
                    adapter_id = next_adapter
                    gpu_id = g
                    sent = True
                    break

            # 2. LOAD if needed
            if not sent:
                cost = env.adapter_vram[next_adapter]
                for g in range(env.num_gpus):
                    if env.free_vram[g] >= cost:
                        action = 0 * A + next_adapter * G + g
                        action_type_name = "LOAD"
                        adapter_id = next_adapter
                        gpu_id = g
                        sent = True
                        break

            # 3. EVICT fallback
            if not sent:
                for g in range(env.num_gpus):
                    for a in range(env.num_adapters):
                        if loaded[g, a] == 1:
                            action = 1 * A + a * G + g
                            action_type_name = "EVICT"
                            adapter_id = a
                            gpu_id = g
                            sent = True
                            break
                    if sent:
                        break

        # Execute action
        prev_queue_len = len(env.queue) if hasattr(env, 'queue') else state[2]
        prev_vram = state[1]

        state, reward, done, _, _ = env.step(action)
        total_reward += reward

        # Calculate what changed
        new_queue_len = len(env.queue) if hasattr(env, 'queue') else state[2]
        queue_change = prev_queue_len - new_queue_len

        # Log this step
        print(f"Step {step:3d}: {action_type_name} A{adapter_id}→G{gpu_id} | "
              f"Reward: {reward:+.2f} | "
              f"Queue:{new_queue_len} |"
              f"VRAM: {[f'{v:.2f}' for v in state[1]]}")

        if done:
            print(f"Episode terminated at step {step}")
            break

    print("-" * 60)
    print(f"Total reward: {total_reward:.2f}")
    print(f"Average reward per step: {total_reward/(step+1):.2f}")

    return total_reward


def load_google_trace(csv_path, max_rows=50000):
    """
    Load Google Cluster Dataset and map collection_id to {0,1,2}
    """

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    # Detect unnamed index column
    sample_df = pd.read_csv(csv_path, nrows=1)
    has_unnamed = 'Unnamed: 0' in sample_df.columns

    print(f"Loading {csv_path}...")
    df = pd.read_csv(csv_path, index_col=0 if has_unnamed else None)

    if max_rows and len(df) > max_rows:
        df = df.head(max_rows).copy()
        print(f"Limited to {max_rows} rows")

    # ---- REQUIRED COLUMNS ----
    required_cols = ['time', 'collection_id', 'assigned_memory']
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"CSV missing required columns: {missing}")

    # ---- MAP collection_id → {0,1,2} ----
    if "collection_id" in df.columns:
        df["collection_id"] = df["collection_id"].astype(int) % 3
    else:
        df["collection_id"] = df.index.astype(int) % 3

    arrivals_by_time = defaultdict(list)

    max_mem = df['assigned_memory'].max()
    if max_mem == 0:
        max_mem = 1.0

    for _, row in df.iterrows():
        t = int(row['time'])

        adapter_id = int(row['collection_id']) % 3

        task = {
            'adapter_id': adapter_id,
            'vram': row['assigned_memory'] / max_mem,
            'collection_id': int(row['collection_id'])
        }

        arrivals_by_time[t].append(task)


    print(f"✓ Loaded {len(df)} tasks across {len(arrivals_by_time)} time steps")
    print(df[['time','collection_type','collection_id','assigned_memory']].head())

    return arrivals_by_time


if __name__ == "__main__":
    CSV_PATH = "/kaggle/input/google-cluster-sample/borg_traces_data.csv"

    print("Loading Google Cluster Dataset...")
    arrivals_by_time = load_google_trace(CSV_PATH, max_rows=5000)
    arrivals = load_google_trace(CSV_PATH)
    sample_t = next(iter(arrivals))
    print(arrivals[sample_t][:5])
    env = AdapterEnv(arrivals_by_time, num_adapters=3, num_gpus=2, max_steps=200)

    test_env_heuristic(env)
    # Train
    print("\nStarting Q-learning...")
    Q_table, rewards ,  epsilon_history= train_q_learning(env, episodes=5000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from collections import deque

def simple_animate_episode(Q, env, max_steps=100):
    """Create a simple animated visualization of a single episode with reward and domain tracking"""

    # Run episode and collect data
    state, _ = env.reset()
    states_history = [state]
    actions_history = []
    rewards_history = []
    cumulative_rewards = [0]  # Start with 0

    done = False
    step = 0

    while not done and step < max_steps:
        if state in Q:
            action = np.argmax(Q[state])
        else:
            action = env.action_space.sample()

        actions_history.append(action)
        state, reward, done, _, _ = env.step(action)
        states_history.append(state)
        rewards_history.append(reward)
        cumulative_rewards.append(cumulative_rewards[-1] + reward)  # Track cumulative
        step += 1

    # Create figure with 2x3 grid (6 plots)
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))

    # 1. VRAM bar chart (top left)
    vram_bars = axes[0, 0].bar(range(env.num_gpus), [0] * env.num_gpus,
                               color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:env.num_gpus])
    axes[0, 0].set_title('GPU VRAM Usage')
    axes[0, 0].set_xlabel('GPU ID')
    axes[0, 0].set_ylabel('Free VRAM')
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].set_xticks(range(env.num_gpus))

    # 2. Queue plot (top middle)
    test_env = AdapterEnv(env.arrivals_by_time, num_adapters=env.num_adapters, num_gpus=env.num_gpus)
    test_state, _ = test_env.reset()
    tracked_queue = [len(test_env.queue)]
    tracked_domains = []  # Track which adapter is needed

    # Replay to track queue and domain
    for i, action in enumerate(actions_history):
        # Record domain before step
        if test_env.queue:
            tracked_domains.append(test_env.queue[0]['adapter_id'])
        else:
            tracked_domains.append(-1)  # No task

        test_state, reward, done, _, _ = test_env.step(action)
        tracked_queue.append(len(test_env.queue))

    # Add final domain state
    if test_env.queue:
        tracked_domains.append(test_env.queue[0]['adapter_id'])
    else:
        tracked_domains.append(-1)

    queue_line, = axes[0, 1].plot([], [], 'r-', linewidth=2, marker='o', markersize=4)
    axes[0, 1].set_title('Queue Length Over Time')
    axes[0, 1].set_xlabel('Step')
    axes[0, 1].set_ylabel('Queue Length')
    axes[0, 1].grid(True, alpha=0.3)
    max_queue = max(tracked_queue) if tracked_queue else 20
    axes[0, 1].set_ylim(0, max_queue * 1.1 if max_queue > 0 else 20)

    # 3. REWARD PLOT (top right) - NEW
    reward_line, = axes[0, 2].plot([], [], 'g-', linewidth=2, marker='s', markersize=4)
    axes[0, 2].set_title('Cumulative Reward')
    axes[0, 2].set_xlabel('Step')
    axes[0, 2].set_ylabel('Total Reward')
    axes[0, 2].grid(True, alpha=0.3)
    min_reward = min(cumulative_rewards)
    max_reward = max(cumulative_rewards)
    reward_range = max_reward - min_reward
    axes[0, 2].set_ylim(min_reward - 0.1*reward_range, max_reward + 0.1*reward_range)

    # 4. Action distribution (bottom left)
    action_types = ['Load', 'Evict', 'Reuse', 'Send']
    action_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    action_counts = [0, 0, 0, 0]
    action_bars = axes[1, 0].bar(action_types, action_counts, color=action_colors)
    axes[1, 0].set_title('Action Types Taken')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].set_ylim(0, max(5, len(actions_history)//4))

    # 5. DOMAIN PLOT (bottom middle) - NEW
    # Show which adapter domain is being requested over time
    domain_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']
    domain_line, = axes[1, 1].plot([], [], 'o-', linewidth=2, markersize=6,
                                   color='purple', alpha=0.7)
    axes[1, 1].set_title('Requested Adapter Domain')
    axes[1, 1].set_xlabel('Step')
    axes[1, 1].set_ylabel('Adapter ID')
    axes[1, 1].set_ylim(-1.5, env.num_adapters - 0.5)
    axes[1, 1].set_yticks(range(-1, env.num_adapters))
    axes[1, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
    axes[1, 1].grid(True, alpha=0.3)

    # 6. Text info (bottom right)
    info_text = axes[1, 2].text(0.05, 0.95, '', fontsize=10,
                                transform=axes[1, 2].transAxes,
                                verticalalignment='top',
                                fontfamily='monospace',
                                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9))
    axes[1, 2].axis('off')
    axes[1, 2].set_title('Step Details')

    fig.suptitle('Adapter Management Agent - Episode Simulation',
                 fontsize=16, fontweight='bold', y=0.98)

    def update(frame):
        if frame >= len(states_history):
            return

        state = states_history[frame]
        loaded_adapters = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)
        free_vram = np.array(state[1])

        # 1. Update VRAM bars
        for i, bar in enumerate(vram_bars):
            if i < len(free_vram):
                bar.set_height(free_vram[i])

        # 2. Update queue plot
        if frame < len(tracked_queue):
            queue_data = tracked_queue[:frame+1]
            queue_line.set_data(range(len(queue_data)), queue_data)
            axes[0, 1].set_xlim(0, max(1, len(states_history)))

        # 3. Update REWARD plot - NEW
        if frame < len(cumulative_rewards):
            reward_data = cumulative_rewards[:frame+1]
            reward_line.set_data(range(len(reward_data)), reward_data)
            axes[0, 2].set_xlim(0, max(1, len(states_history)))

        # 4. Update action counts
        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            if action_type < len(action_counts):
                action_counts[action_type] += 1
                for i, bar in enumerate(action_bars):
                    bar.set_height(action_counts[i])
                max_count = max(action_counts) if action_counts else 1
                axes[1, 0].set_ylim(0, max_count * 1.2)

        # 5. Update DOMAIN plot - NEW
        if frame < len(tracked_domains):
            domain_data = tracked_domains[:frame+1]
            # Use different colors for different domains
            colors = [domain_colors[d % len(domain_colors)] if d >= 0 else 'gray'
                     for d in domain_data]
            domain_line.set_data(range(len(domain_data)), domain_data)
            # Update scatter colors
            axes[1, 1].clear()
            axes[1, 1].set_title('Requested Adapter Domain')
            axes[1, 1].set_xlabel('Step')
            axes[1, 1].set_ylabel('Adapter ID')
            axes[1, 1].set_ylim(-1.5, env.num_adapters - 0.5)
            axes[1, 1].set_yticks(range(-1, env.num_adapters))
            axes[1, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
            axes[1, 1].grid(True, alpha=0.3)
            axes[1, 1].set_xlim(0, max(1, len(states_history)))

            # Plot with colors
            for i, (x, y) in enumerate(zip(range(len(domain_data)), domain_data)):
                color = domain_colors[y % len(domain_colors)] if y >= 0 else 'gray'
                axes[1, 1].plot(x, y, 'o', color=color, markersize=8, alpha=0.7)
                if i > 0:
                    prev_y = domain_data[i-1]
                    axes[1, 1].plot([i-1, i], [prev_y, y], 'k-', alpha=0.3, linewidth=1)

        # 6. Update info text
        info_str = f"Step: {frame}\n"
        info_str += f"Free VRAM: {[f'{v:.2f}' for v in free_vram]}\n"
        if frame < len(tracked_queue):
            info_str += f"Queue: {tracked_queue[frame]}\n"
        if frame < len(tracked_domains) and tracked_domains[frame] >= 0:
            info_str += f"Need: Adapter {tracked_domains[frame]}\n"

        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            rem = action % (env.num_adapters * env.num_gpus)
            adapter_id = rem // env.num_gpus
            gpu_id = rem % env.num_gpus

            action_names = ['Load', 'Evict', 'Reuse', 'Send']
            info_str += f"\nLast: {action_names[action_type]} A{adapter_id}→G{gpu_id}\n"
            info_str += f"Reward: {rewards_history[frame-1]:+.2f}\n"
            info_str += f"Total: {cumulative_rewards[frame]:.2f}"

        info_text.set_text(info_str)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        return queue_line, reward_line, domain_line, *vram_bars, *action_bars, info_text

    anim = FuncAnimation(fig, update, frames=len(states_history),
                        interval=600, blit=False, repeat=False)

    return HTML(anim.to_jshtml())
print("\n  Generating animation...")
animation = simple_animate_episode(Q_table, env, max_steps=100)
display(animation)

In [ ]:
#heuristics results:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def simple_animate_episode_heuristic(env, max_steps=30):
    """Create animation for heuristic (greedy) policy results"""

    # Run episode with heuristic and collect data
    state, _ = env.reset()
    states_history = [state]
    actions_history = []
    rewards_history = []
    cumulative_rewards = [0]
    action_names_history = []  # Store action names for display

    A = env.num_adapters * env.num_gpus
    G = env.num_gpus

    done = False
    step = 0

    while not done and step < max_steps:
        # Heuristic logic (same as test_env_heuristic)
        action = 0
        action_type_name = "WAIT"
        adapter_id = 0
        gpu_id = 0

        if env.queue:
            next_adapter = env.queue[0]['adapter_id']
            loaded = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)

            # 1. SEND if possible
            sent = False
            for g in range(env.num_gpus):
                if loaded[g, next_adapter] == 1:
                    action = 3 * A + next_adapter * G + g
                    action_type_name = "SEND"
                    adapter_id = next_adapter
                    gpu_id = g
                    sent = True
                    break

            # 2. LOAD if needed
            if not sent:
                cost = env.adapter_vram[next_adapter]
                for g in range(env.num_gpus):
                    if env.free_vram[g] >= cost:
                        action = 0 * A + next_adapter * G + g
                        action_type_name = "LOAD"
                        adapter_id = next_adapter
                        gpu_id = g
                        sent = True
                        break

            # 3. EVICT fallback
            if not sent:
                for g in range(env.num_gpus):
                    for a in range(env.num_adapters):
                        if loaded[g, a] == 1:
                            action = 1 * A + a * G + g
                            action_type_name = "EVICT"
                            adapter_id = a
                            gpu_id = g
                            sent = True
                            break
                    if sent:
                        break

        # Record action info
        actions_history.append(action)
        action_names_history.append((action_type_name, adapter_id, gpu_id))

        # Step environment
        state, reward, done, _, _ = env.step(action)
        states_history.append(state)
        rewards_history.append(reward)
        cumulative_rewards.append(cumulative_rewards[-1] + reward)
        step += 1

    # Track queue and domain history (same as Q-learning version)
    test_env = AdapterEnv(env.arrivals_by_time, num_adapters=env.num_adapters, num_gpus=env.num_gpus)
    test_state, _ = test_env.reset()
    tracked_queue = [len(test_env.queue)]
    tracked_domains = []

    for i, action in enumerate(actions_history):
        if test_env.queue:
            tracked_domains.append(test_env.queue[0]['adapter_id'])
        else:
            tracked_domains.append(-1)
        test_state, _, done, _, _ = test_env.step(action)
        tracked_queue.append(len(test_env.queue))

    if test_env.queue:
        tracked_domains.append(test_env.queue[0]['adapter_id'])
    else:
        tracked_domains.append(-1)

    # Create figure with 2x3 grid
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))

    # 1. VRAM bar chart (top left)
    vram_bars = axes[0, 0].bar(range(env.num_gpus), [0] * env.num_gpus,
                               color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:env.num_gpus])
    axes[0, 0].set_title('GPU VRAM Usage (Heuristic)')
    axes[0, 0].set_xlabel('GPU ID')
    axes[0, 0].set_ylabel('Free VRAM')
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].set_xticks(range(env.num_gpus))

    # 2. Queue plot (top middle)
    queue_line, = axes[0, 1].plot([], [], 'r-', linewidth=2, marker='o', markersize=4)
    axes[0, 1].set_title('Queue Length Over Time')
    axes[0, 1].set_xlabel('Step')
    axes[0, 1].set_ylabel('Queue Length')
    axes[0, 1].grid(True, alpha=0.3)
    max_queue = max(tracked_queue) if tracked_queue else 20
    axes[0, 1].set_ylim(0, max_queue * 1.1 if max_queue > 0 else 20)

    # 3. REWARD PLOT (top right)
    reward_line, = axes[0, 2].plot([], [], 'g-', linewidth=2, marker='s', markersize=4)
    axes[0, 2].set_title('Cumulative Reward (Heuristic)')
    axes[0, 2].set_xlabel('Step')
    axes[0, 2].set_ylabel('Total Reward')
    axes[0, 2].grid(True, alpha=0.3)
    min_reward = min(cumulative_rewards)
    max_reward = max(cumulative_rewards)
    reward_range = max_reward - min_reward
    axes[0, 2].set_ylim(min_reward - 0.1*reward_range, max_reward + 0.1*reward_range)

    # 4. Action distribution (bottom left)
    action_types = ['Load', 'Evict', 'Reuse', 'Send']
    action_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    action_counts = [0, 0, 0, 0]
    action_bars = axes[1, 0].bar(action_types, action_counts, color=action_colors)
    axes[1, 0].set_title('Heuristic Action Types')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].set_ylim(0, max(5, len(actions_history)//4))

    # 5. DOMAIN PLOT (bottom middle)
    domain_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']
    domain_line, = axes[1, 1].plot([], [], 'o-', linewidth=2, markersize=6,
                                   color='purple', alpha=0.7)
    axes[1, 1].set_title('Requested Adapter Domain')
    axes[1, 1].set_xlabel('Step')
    axes[1, 1].set_ylabel('Adapter ID')
    axes[1, 1].set_ylim(-1.5, env.num_adapters - 0.5)
    axes[1, 1].set_yticks(range(-1, env.num_adapters))
    axes[1, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
    axes[1, 1].grid(True, alpha=0.3)

    # 6. Text info (bottom right)
    info_text = axes[1, 2].text(0.05, 0.95, '', fontsize=10,
                                transform=axes[1, 2].transAxes,
                                verticalalignment='top',
                                fontfamily='monospace',
                                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))
    axes[1, 2].axis('off')
    axes[1, 2].set_title('Heuristic Decision Log')

    fig.suptitle('Adapter Management - Heuristic (Greedy) Policy',
                 fontsize=16, fontweight='bold', y=0.98)

    def update(frame):
        if frame >= len(states_history):
            return

        state = states_history[frame]
        loaded_adapters = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)
        free_vram = np.array(state[1])

        # 1. Update VRAM bars
        for i, bar in enumerate(vram_bars):
            if i < len(free_vram):
                bar.set_height(free_vram[i])

        # 2. Update queue plot
        if frame < len(tracked_queue):
            queue_data = tracked_queue[:frame+1]
            queue_line.set_data(range(len(queue_data)), queue_data)
            axes[0, 1].set_xlim(0, max(1, len(states_history)))

        # 3. Update REWARD plot
        if frame < len(cumulative_rewards):
            reward_data = cumulative_rewards[:frame+1]
            reward_line.set_data(range(len(reward_data)), reward_data)
            axes[0, 2].set_xlim(0, max(1, len(states_history)))

        # 4. Update action counts
        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            if action_type < len(action_counts):
                action_counts[action_type] += 1
                for i, bar in enumerate(action_bars):
                    bar.set_height(action_counts[i])
                max_count = max(action_counts) if action_counts else 1
                axes[1, 0].set_ylim(0, max_count * 1.2)

        # 5. Update DOMAIN plot
        if frame < len(tracked_domains):
            domain_data = tracked_domains[:frame+1]
            axes[1, 1].clear()
            axes[1, 1].set_title('Requested Adapter Domain')
            axes[1, 1].set_xlabel('Step')
            axes[1, 1].set_ylabel('Adapter ID')
            axes[1, 1].set_ylim(-1.5, env.num_adapters - 0.5)
            axes[1, 1].set_yticks(range(-1, env.num_adapters))
            axes[1, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
            axes[1, 1].grid(True, alpha=0.3)
            axes[1, 1].set_xlim(0, max(1, len(states_history)))

            for i, (x, y) in enumerate(zip(range(len(domain_data)), domain_data)):
                color = domain_colors[y % len(domain_colors)] if y >= 0 else 'gray'
                axes[1, 1].plot(x, y, 'o', color=color, markersize=8, alpha=0.7)
                if i > 0:
                    prev_y = domain_data[i-1]
                    axes[1, 1].plot([i-1, i], [prev_y, y], 'k-', alpha=0.3, linewidth=1)

        # 6. Update info text with heuristic decision
        info_str = f"Step: {frame}\n"
        info_str += f"Free VRAM: {[f'{v:.2f}' for v in free_vram]}\n"
        if frame < len(tracked_queue):
            info_str += f"Queue: {tracked_queue[frame]}\n"
        if frame < len(tracked_domains) and tracked_domains[frame] >= 0:
            info_str += f"Need: Adapter {tracked_domains[frame]}\n"

        if frame > 0 and frame-1 < len(action_names_history):
            action_name, adapter_id, gpu_id = action_names_history[frame-1]
            info_str += f"\nHeuristic: {action_name}\n"
            info_str += f"  A{adapter_id} → GPU{gpu_id}\n"
            info_str += f"Reward: {rewards_history[frame-1]:+.2f}\n"
            info_str += f"Total: {cumulative_rewards[frame]:.2f}"

        info_text.set_text(info_str)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        return queue_line, reward_line, domain_line, *vram_bars, *action_bars, info_text

    anim = FuncAnimation(fig, update, frames=len(states_history),
                        interval=600, blit=False, repeat=False)

    return HTML(anim.to_jshtml())

# === USAGE ===
# After running test_env_heuristic or with your env:
print("\n Generating heuristic animation...")
animation_heuristic = simple_animate_episode_heuristic(env, max_steps=50)
display(animation_heuristic)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from collections import deque

def simple_animate_episode(Q, env, max_steps=100):
    """Create a simple animated visualization of a single episode with reward and domain tracking"""

    # Run episode and collect data
    state, _ = env.reset()
    states_history = [state]
    actions_history = []
    rewards_history = []
    cumulative_rewards = [0]  # Start with 0

    done = False
    step = 0

    while not done and step < max_steps:
        if state in Q:
            action = np.argmax(Q[state])
        else:
            action = env.action_space.sample()

        actions_history.append(action)
        state, reward, done, _, _ = env.step(action)
        states_history.append(state)
        rewards_history.append(reward)
        cumulative_rewards.append(cumulative_rewards[-1] + reward)  # Track cumulative
        step += 1

    # Create figure with 3x2 grid (6 plots) - Removed empty diagrams
    fig, axes = plt.subplots(3, 2, figsize=(15, 14))  # Increased height

    # 1. VRAM bar chart (row 1, col 1)
    vram_bars = axes[0, 0].bar(range(env.num_gpus), [0] * env.num_gpus,
                               color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:env.num_gpus])
    axes[0, 0].set_title('GPU VRAM Usage', fontsize=12, pad=10)
    axes[0, 0].set_xlabel('GPU ID', fontsize=10)
    axes[0, 0].set_ylabel('Free VRAM', fontsize=10)
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].set_xticks(range(env.num_gpus))

    # 2. Queue plot (row 1, col 2)
    test_env = AdapterEnv(env.arrivals_by_time, num_adapters=env.num_adapters, num_gpus=env.num_gpus)
    test_state, _ = test_env.reset()
    tracked_queue = [len(test_env.queue)]
    tracked_domains = []  # Track which adapter is needed

    # Replay to track queue and domain
    for i, action in enumerate(actions_history):
        # Record domain before step
        if test_env.queue:
            tracked_domains.append(test_env.queue[0]['adapter_id'])
        else:
            tracked_domains.append(-1)  # No task

        test_state, reward, done, _, _ = test_env.step(action)
        tracked_queue.append(len(test_env.queue))

    # Add final domain state
    if test_env.queue:
        tracked_domains.append(test_env.queue[0]['adapter_id'])
    else:
        tracked_domains.append(-1)

    queue_line, = axes[0, 1].plot([], [], 'r-', linewidth=2, marker='o', markersize=4)
    axes[0, 1].set_title('Queue Length Over Time', fontsize=12, pad=10)
    axes[0, 1].set_xlabel('Step', fontsize=10)
    axes[0, 1].set_ylabel('Queue Length', fontsize=10)
    axes[0, 1].grid(True, alpha=0.3)
    max_queue = max(tracked_queue) if tracked_queue else 20
    axes[0, 1].set_ylim(0, max_queue * 1.1 if max_queue > 0 else 20)

    # 3. Cumulative Reward plot (row 2, col 1)
    cumulative_line, = axes[1, 0].plot([], [], 'g-', linewidth=2, marker='s', markersize=4)
    axes[1, 0].set_title('Cumulative Reward', fontsize=12, pad=10)
    axes[1, 0].set_xlabel('Step', fontsize=10)
    axes[1, 0].set_ylabel('Total Reward', fontsize=10)
    axes[1, 0].grid(True, alpha=0.3)

    # 4. REWARD PER STEP plot (row 2, col 2)
    step_reward_line, = axes[1, 1].plot([], [], 'b-', linewidth=2, marker='D', markersize=4)
    axes[1, 1].set_title('Reward Per Step', fontsize=12, pad=10)
    axes[1, 1].set_xlabel('Step', fontsize=10)
    axes[1, 1].set_ylabel('Step Reward', fontsize=10)
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)

    # Color code for positive/negative rewards
    step_reward_pos, = axes[1, 1].plot([], [], 'go', markersize=6, alpha=0.7, label='Positive')
    step_reward_neg, = axes[1, 1].plot([], [], 'ro', markersize=6, alpha=0.7, label='Negative')
    axes[1, 1].legend(loc='upper right', fontsize=8)

    # 5. Action distribution (row 3, col 1)
    action_types = ['Load', 'Evict', 'Reuse', 'Send']
    action_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    action_counts = [0, 0, 0, 0]
    action_bars = axes[2, 0].bar(action_types, action_counts, color=action_colors)
    axes[2, 0].set_title('Action Types Taken', fontsize=12, pad=10)
    axes[2, 0].set_xlabel('Action Type', fontsize=10)
    axes[2, 0].set_ylabel('Count', fontsize=10)
    axes[2, 0].set_ylim(0, max(5, len(actions_history)//4))

    # 6. DOMAIN PLOT - Requested adapters (row 3, col 2)
    domain_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']
    requested_line, = axes[2, 1].plot([], [], 'o-', linewidth=2, markersize=6,
                                     color='purple', alpha=0.7, label='Requested')
    axes[2, 1].set_title('Requested Adapter', fontsize=12, pad=10)
    axes[2, 1].set_xlabel('Step', fontsize=10)
    axes[2, 1].set_ylabel('Adapter ID', fontsize=10)
    axes[2, 1].set_ylim(-1.5, env.num_adapters - 0.5)
    axes[2, 1].set_yticks(range(-1, env.num_adapters))
    axes[2, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
    axes[2, 1].grid(True, alpha=0.3)

    # 7. Text info - Create a separate axis on the right side
    fig.subplots_adjust(right=0.85)  # Make room for text on the right
    info_ax = fig.add_axes([0.87, 0.1, 0.12, 0.8])  # [left, bottom, width, height]
    info_ax.axis('off')
    info_text = info_ax.text(0.02, 0.98, '', fontsize=9,
                             verticalalignment='top',
                             fontfamily='monospace',
                             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9))

    fig.suptitle('Adapter Management Agent - Episode Simulation',
                 fontsize=16, fontweight='bold', y=0.98)

    def update(frame):
        if frame >= len(states_history):
            return

        state = states_history[frame]
        loaded_adapters = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)
        free_vram = np.array(state[1])

        # 1. Update VRAM bars
        for i, bar in enumerate(vram_bars):
            if i < len(free_vram):
                bar.set_height(free_vram[i])

        # 2. Update queue plot
        if frame < len(tracked_queue):
            queue_data = tracked_queue[:frame+1]
            queue_line.set_data(range(len(queue_data)), queue_data)
            axes[0, 1].set_xlim(0, max(1, len(states_history)))

        # 3. Update Cumulative Reward plot
        if frame < len(cumulative_rewards):
            cumulative_data = cumulative_rewards[:frame+1]
            cumulative_line.set_data(range(len(cumulative_data)), cumulative_data)
            axes[1, 0].set_xlim(0, max(1, len(states_history)))

            # Auto-scale y-axis for cumulative reward
            if len(cumulative_data) > 0:
                min_val = min(cumulative_data)
                max_val = max(cumulative_data)
                if min_val != max_val:
                    axes[1, 0].set_ylim(min_val - 0.1*(max_val-min_val),
                                       max_val + 0.1*(max_val-min_val))

        # 4. Update REWARD PER STEP plot
        if frame > 0:
            # Plot all step rewards
            step_data = rewards_history[:frame]
            x_data = range(1, frame+1)

            # Clear and redraw
            axes[1, 1].clear()
            axes[1, 1].set_title('Reward Per Step', fontsize=12, pad=10)
            axes[1, 1].set_xlabel('Step', fontsize=10)
            axes[1, 1].set_ylabel('Step Reward', fontsize=10)
            axes[1, 1].grid(True, alpha=0.3)
            axes[1, 1].axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)

            # Separate positive and negative rewards
            pos_x, pos_y = [], []
            neg_x, neg_y = [], []

            for i, reward in enumerate(step_data):
                if reward >= 0:
                    pos_x.append(i+1)
                    pos_y.append(reward)
                else:
                    neg_x.append(i+1)
                    neg_y.append(reward)

            # Plot with different colors
            if pos_x:
                axes[1, 1].plot(pos_x, pos_y, 'go', markersize=6, alpha=0.7, label='Positive')
            if neg_x:
                axes[1, 1].plot(neg_x, neg_y, 'ro', markersize=6, alpha=0.7, label='Negative')

            # Add connecting lines
            if len(step_data) > 1:
                axes[1, 1].plot(x_data, step_data, 'b-', alpha=0.3, linewidth=1)

            axes[1, 1].set_xlim(0.5, max(1.5, frame+0.5))
            if step_data:
                min_reward = min(step_data)
                max_reward = max(step_data)
                reward_range = max_reward - min_reward
                if reward_range > 0:
                    axes[1, 1].set_ylim(min_reward - 0.1*reward_range, max_reward + 0.1*reward_range)

            axes[1, 1].legend(loc='upper right', fontsize=8)

        # 5. Update action counts
        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            if action_type < len(action_counts):
                action_counts[action_type] += 1
                for i, bar in enumerate(action_bars):
                    bar.set_height(action_counts[i])
                max_count = max(action_counts) if action_counts else 1
                axes[2, 0].set_ylim(0, max_count * 1.2)

        # 6. Update DOMAIN plot (Requested adapters)
        if frame < len(tracked_domains):
            domain_data = tracked_domains[:frame+1]
            requested_line.set_data(range(len(domain_data)), domain_data)
            axes[2, 1].set_xlim(0, max(1, len(states_history)))

            # Color code points
            axes[2, 1].clear()
            axes[2, 1].set_title('Requested Adapter', fontsize=12, pad=10)
            axes[2, 1].set_xlabel('Step', fontsize=10)
            axes[2, 1].set_ylabel('Adapter ID', fontsize=10)
            axes[2, 1].set_ylim(-1.5, env.num_adapters - 0.5)
            axes[2, 1].set_yticks(range(-1, env.num_adapters))
            axes[2, 1].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
            axes[2, 1].grid(True, alpha=0.3)
            axes[2, 1].set_xlim(0, max(1, len(states_history)))

            for i, adapter in enumerate(domain_data):
                color = domain_colors[adapter % len(domain_colors)] if adapter >= 0 else 'gray'
                axes[2, 1].plot(i, adapter, 'o', color=color, markersize=8, alpha=0.7)
                if i > 0:
                    prev_adapter = domain_data[i-1]
                    axes[2, 1].plot([i-1, i], [prev_adapter, adapter], 'k-', alpha=0.3, linewidth=1)

        # 7. Update info text
        info_str = f"Step: {frame}\n"
        info_str += f"Free VRAM: {[f'{v:.2f}' for v in free_vram]}\n"
        if frame < len(tracked_queue):
            info_str += f"Queue: {tracked_queue[frame]}\n"
        if frame < len(tracked_domains) and tracked_domains[frame] >= 0:
            info_str += f"Need: Adapter {tracked_domains[frame]}\n"

        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            rem = action % (env.num_adapters * env.num_gpus)
            adapter_id = rem // env.num_gpus
            gpu_id = rem % env.num_gpus

            action_names = ['Load', 'Evict', 'Reuse', 'Send']
            info_str += f"\nLast Action:\n"
            info_str += f"  {action_names[action_type]} A{adapter_id}→G{gpu_id}\n"
            info_str += f"  Step Reward: {rewards_history[frame-1]:+.2f}\n"
            info_str += f"  Total: {cumulative_rewards[frame]:.2f}"

            # Add reward statistics
            if frame > 1:
                avg_reward = np.mean(rewards_history[:frame])
                pos_rewards = sum(1 for r in rewards_history[:frame] if r > 0)
                neg_rewards = sum(1 for r in rewards_history[:frame] if r < 0)
                info_str += f"\n\nReward Stats:\n"
                info_str += f"  Avg/step: {avg_reward:.2f}\n"
                info_str += f"  Positive: {pos_rewards}\n"
                info_str += f"  Negative: {neg_rewards}"

        info_text.set_text(info_str)

        # Adjust layout to prevent overlap
        plt.tight_layout(rect=[0, 0, 0.85, 0.96])  # Leave right 15% for text

        return (queue_line, cumulative_line, step_reward_line,
                requested_line, *vram_bars, info_text)

    anim = FuncAnimation(fig, update, frames=len(states_history),
                        interval=600, blit=False, repeat=False)

    return HTML(anim.to_jshtml())

print("\nGenerating animation...")
animation = simple_animate_episode(Q_table, env, max_steps=100)
display(animation)

In [ ]:
#heuristics results:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def simple_animate_episode_heuristic(env, max_steps=100):
    """Create animation for heuristic (greedy) policy results"""

    # Run episode with heuristic and collect data
    state, _ = env.reset()
    states_history = [state]
    actions_history = []
    rewards_history = []
    cumulative_rewards = [0]
    action_names_history = []  # Store action names for display

    A = env.num_adapters * env.num_gpus
    G = env.num_gpus

    done = False
    step = 0

    while not done and step < max_steps:
        # Heuristic logic (same as test_env_heuristic)
        action = 0
        action_type_name = "WAIT"
        adapter_id = 0
        gpu_id = 0

        if env.queue:
            next_adapter = env.queue[0]['adapter_id']
            loaded = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)

            # 1. SEND if possible
            sent = False
            for g in range(env.num_gpus):
                if loaded[g, next_adapter] == 1:
                    action = 3 * A + next_adapter * G + g
                    action_type_name = "SEND"
                    adapter_id = next_adapter
                    gpu_id = g
                    sent = True
                    break

            # 2. LOAD if needed
            if not sent:
                cost = env.adapter_vram[next_adapter]
                for g in range(env.num_gpus):
                    if env.free_vram[g] >= cost:
                        action = 0 * A + next_adapter * G + g
                        action_type_name = "LOAD"
                        adapter_id = next_adapter
                        gpu_id = g
                        sent = True
                        break

            # 3. EVICT fallback
            if not sent:
                for g in range(env.num_gpus):
                    for a in range(env.num_adapters):
                        if loaded[g, a] == 1:
                            action = 1 * A + a * G + g
                            action_type_name = "EVICT"
                            adapter_id = a
                            gpu_id = g
                            sent = True
                            break
                    if sent:
                        break

        # Record action info
        actions_history.append(action)
        action_names_history.append((action_type_name, adapter_id, gpu_id))

        # Step environment
        state, reward, done, _, _ = env.step(action)
        states_history.append(state)
        rewards_history.append(reward)
        cumulative_rewards.append(cumulative_rewards[-1] + reward)
        step += 1

    # Track queue and domain history (same as Q-learning version)
    test_env = AdapterEnv(env.arrivals_by_time, num_adapters=env.num_adapters, num_gpus=env.num_gpus)
    test_state, _ = test_env.reset()
    tracked_queue = [len(test_env.queue)]
    tracked_domains = []

    for i, action in enumerate(actions_history):
        if test_env.queue:
            tracked_domains.append(test_env.queue[0]['adapter_id'])
        else:
            tracked_domains.append(-1)
        test_state, _, done, _, _ = test_env.step(action)
        tracked_queue.append(len(test_env.queue))

    if test_env.queue:
        tracked_domains.append(test_env.queue[0]['adapter_id'])
    else:
        tracked_domains.append(-1)

    # Create figure with 3x3 grid to accommodate all plots including Reward Per Step
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))

    # 1. VRAM bar chart (row 1, col 1)
    vram_bars = axes[0, 0].bar(range(env.num_gpus), [0] * env.num_gpus,
                               color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:env.num_gpus])
    axes[0, 0].set_title('GPU VRAM Usage (Heuristic)')
    axes[0, 0].set_xlabel('GPU ID')
    axes[0, 0].set_ylabel('Free VRAM')
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].set_xticks(range(env.num_gpus))

    # 2. Queue plot (row 1, col 2)
    queue_line, = axes[0, 1].plot([], [], 'r-', linewidth=2, marker='o', markersize=4)
    axes[0, 1].set_title('Queue Length Over Time')
    axes[0, 1].set_xlabel('Step')
    axes[0, 1].set_ylabel('Queue Length')
    axes[0, 1].grid(True, alpha=0.3)
    max_queue = max(tracked_queue) if tracked_queue else 20
    axes[0, 1].set_ylim(0, max_queue * 1.1 if max_queue > 0 else 20)

    # 3. REWARD PER STEP plot (row 1, col 3) - NEW!
    step_reward_line, = axes[0, 2].plot([], [], 'b-', linewidth=2, marker='D', markersize=4)
    axes[0, 2].set_title('Reward Per Step (Heuristic)')
    axes[0, 2].set_xlabel('Step')
    axes[0, 2].set_ylabel('Step Reward')
    axes[0, 2].grid(True, alpha=0.3)
    axes[0, 2].axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)

    # Color code for positive/negative rewards
    step_reward_pos, = axes[0, 2].plot([], [], 'go', markersize=6, alpha=0.7, label='Positive')
    step_reward_neg, = axes[0, 2].plot([], [], 'ro', markersize=6, alpha=0.7, label='Negative')
    axes[0, 2].legend(loc='upper right', fontsize=8)

    # 4. CUMULATIVE REWARD plot (row 2, col 1)
    cumulative_line, = axes[1, 0].plot([], [], 'g-', linewidth=2, marker='s', markersize=4)
    axes[1, 0].set_title('Cumulative Reward (Heuristic)')
    axes[1, 0].set_xlabel('Step')
    axes[1, 0].set_ylabel('Total Reward')
    axes[1, 0].grid(True, alpha=0.3)

    # 5. Action distribution (row 2, col 2)
    action_types = ['Load', 'Evict', 'Reuse', 'Send']
    action_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    action_counts = [0, 0, 0, 0]
    action_bars = axes[1, 1].bar(action_types, action_counts, color=action_colors)
    axes[1, 1].set_title('Heuristic Action Types')
    axes[1, 1].set_xlabel('Action Type')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_ylim(0, max(5, len(actions_history)//4))

    # 6. DOMAIN PLOT (row 2, col 3)
    domain_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']
    domain_line, = axes[1, 2].plot([], [], 'o-', linewidth=2, markersize=6,
                                   color='purple', alpha=0.7)
    axes[1, 2].set_title('Requested Adapter Domain')
    axes[1, 2].set_xlabel('Step')
    axes[1, 2].set_ylabel('Adapter ID')
    axes[1, 2].set_ylim(-1.5, env.num_adapters - 0.5)
    axes[1, 2].set_yticks(range(-1, env.num_adapters))
    axes[1, 2].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
    axes[1, 2].grid(True, alpha=0.3)

    # 7. Text info (row 3, col 1) - spanning 3 columns
    info_ax = axes[2, 0]
    info_text = info_ax.text(0.05, 0.95, '', fontsize=9,
                             transform=info_ax.transAxes,
                             verticalalignment='top',
                             fontfamily='monospace',
                             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))
    info_ax.axis('off')
    info_ax.set_title('Heuristic Decision Log')

    # Hide empty subplots (row 3, col 2 and col 3)
    axes[2, 1].axis('off')
    axes[2, 2].axis('off')

    fig.suptitle('Adapter Management - Heuristic (Greedy) Policy',
                 fontsize=16, fontweight='bold', y=0.98)

    def update(frame):
        if frame >= len(states_history):
            return

        state = states_history[frame]
        loaded_adapters = np.array(state[0]).reshape(env.num_gpus, env.num_adapters)
        free_vram = np.array(state[1])

        # 1. Update VRAM bars
        for i, bar in enumerate(vram_bars):
            if i < len(free_vram):
                bar.set_height(free_vram[i])

        # 2. Update queue plot
        if frame < len(tracked_queue):
            queue_data = tracked_queue[:frame+1]
            queue_line.set_data(range(len(queue_data)), queue_data)
            axes[0, 1].set_xlim(0, max(1, len(states_history)))

        # 3. Update REWARD PER STEP plot
        if frame > 0:
            # Plot all step rewards
            step_data = rewards_history[:frame]
            x_data = range(1, frame+1)

            # Clear and redraw
            axes[0, 2].clear()
            axes[0, 2].set_title('Reward Per Step (Heuristic)')
            axes[0, 2].set_xlabel('Step')
            axes[0, 2].set_ylabel('Step Reward')
            axes[0, 2].grid(True, alpha=0.3)
            axes[0, 2].axhline(y=0, color='gray', linestyle='-', alpha=0.3, linewidth=1)

            # Separate positive and negative rewards
            pos_x, pos_y = [], []
            neg_x, neg_y = [], []

            for i, reward in enumerate(step_data):
                if reward >= 0:
                    pos_x.append(i+1)
                    pos_y.append(reward)
                else:
                    neg_x.append(i+1)
                    neg_y.append(reward)

            # Plot with different colors
            if pos_x:
                axes[0, 2].plot(pos_x, pos_y, 'go', markersize=6, alpha=0.7, label='Positive')
            if neg_x:
                axes[0, 2].plot(neg_x, neg_y, 'ro', markersize=6, alpha=0.7, label='Negative')

            # Add connecting lines
            if len(step_data) > 1:
                axes[0, 2].plot(x_data, step_data, 'b-', alpha=0.3, linewidth=1)

            axes[0, 2].set_xlim(0.5, max(1.5, frame+0.5))
            if step_data:
                min_reward = min(step_data)
                max_reward = max(step_data)
                reward_range = max_reward - min_reward
                if reward_range > 0:
                    axes[0, 2].set_ylim(min_reward - 0.1*reward_range, max_reward + 0.1*reward_range)

            axes[0, 2].legend(loc='upper right', fontsize=8)

        # 4. Update CUMULATIVE REWARD plot
        if frame < len(cumulative_rewards):
            cumulative_data = cumulative_rewards[:frame+1]
            cumulative_line.set_data(range(len(cumulative_data)), cumulative_data)
            axes[1, 0].set_xlim(0, max(1, len(states_history)))

            # Auto-scale y-axis for cumulative reward
            if len(cumulative_data) > 0:
                min_val = min(cumulative_data)
                max_val = max(cumulative_data)
                if min_val != max_val:
                    axes[1, 0].set_ylim(min_val - 0.1*(max_val-min_val),
                                       max_val + 0.1*(max_val-min_val))

        # 5. Update action counts
        if frame > 0 and frame-1 < len(actions_history):
            action = actions_history[frame-1]
            action_type = action // (env.num_adapters * env.num_gpus)
            if action_type < len(action_counts):
                action_counts[action_type] += 1
                for i, bar in enumerate(action_bars):
                    bar.set_height(action_counts[i])
                max_count = max(action_counts) if action_counts else 1
                axes[1, 1].set_ylim(0, max_count * 1.2)

        # 6. Update DOMAIN plot
        if frame < len(tracked_domains):
            domain_data = tracked_domains[:frame+1]
            axes[1, 2].clear()
            axes[1, 2].set_title('Requested Adapter Domain')
            axes[1, 2].set_xlabel('Step')
            axes[1, 2].set_ylabel('Adapter ID')
            axes[1, 2].set_ylim(-1.5, env.num_adapters - 0.5)
            axes[1, 2].set_yticks(range(-1, env.num_adapters))
            axes[1, 2].set_yticklabels(['None'] + [f'A{i}' for i in range(env.num_adapters)])
            axes[1, 2].grid(True, alpha=0.3)
            axes[1, 2].set_xlim(0, max(1, len(states_history)))

            for i, (x, y) in enumerate(zip(range(len(domain_data)), domain_data)):
                color = domain_colors[y % len(domain_colors)] if y >= 0 else 'gray'
                axes[1, 2].plot(x, y, 'o', color=color, markersize=8, alpha=0.7)
                if i > 0:
                    prev_y = domain_data[i-1]
                    axes[1, 2].plot([i-1, i], [prev_y, y], 'k-', alpha=0.3, linewidth=1)

        # 7. Update info text with heuristic decision
        info_str = f"Step: {frame}\n"
        info_str += f"Free VRAM: {[f'{v:.2f}' for v in free_vram]}\n"
        if frame < len(tracked_queue):
            info_str += f"Queue: {tracked_queue[frame]}\n"
        if frame < len(tracked_domains) and tracked_domains[frame] >= 0:
            info_str += f"Need: Adapter {tracked_domains[frame]}\n"

        if frame > 0 and frame-1 < len(action_names_history):
            action_name, adapter_id, gpu_id = action_names_history[frame-1]
            info_str += f"\nHeuristic: {action_name}\n"
            info_str += f"  A{adapter_id} → GPU{gpu_id}\n"
            info_str += f"Step Reward: {rewards_history[frame-1]:+.2f}\n"
            info_str += f"Total: {cumulative_rewards[frame]:.2f}\n"

            # Add reward statistics
            if frame > 1:
                avg_reward = np.mean(rewards_history[:frame])
                pos_rewards = sum(1 for r in rewards_history[:frame] if r > 0)
                neg_rewards = sum(1 for r in rewards_history[:frame] if r < 0)
                info_str += f"\nReward Stats:\n"
                info_str += f"  Avg/step: {avg_reward:.2f}\n"
                info_str += f"  Positive: {pos_rewards}\n"
                info_str += f"  Negative: {neg_rewards}"

        info_text.set_text(info_str)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        return (queue_line, step_reward_line, cumulative_line, domain_line,
                *vram_bars, *action_bars, info_text)

    anim = FuncAnimation(fig, update, frames=len(states_history),
                        interval=600, blit=False, repeat=False)

    return HTML(anim.to_jshtml())

# === USAGE ===
# After running test_env_heuristic or with your env:
print("\nGenerating heuristic animation...")
animation_heuristic = simple_animate_episode_heuristic(env, max_steps=100)
display(animation_heuristic)